# Описание обучения

### Данные

1. Берутся только id героев как фичи

### Обучение

1. Для **ML подходов:** \
Конкатинируются в вектор длины $ 2 * len(unique(id)) $ \
Для Radiant - $ len(unique(id)) $ \
Для Dire - $ len(unique(id)) $

2. Для **DL подходов:** ?

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/ankamenskiy/SmartDota/')

In [60]:
import pickle
import dataclasses

import pandas as pd
import numpy as np
import torch

from src.data.api.download import ProMatchesDataloader

## Читаем данные

In [33]:
data_path = '/home/ankamenskiy/SmartDota/cache/pro_1000'

loader = ProMatchesDataloader()
loader.load(path=data_path)

data = loader.data

None


## Готовим датасет

0 - Radiant \
1 - Dire

In [73]:
pb = [elem.picks_bans for elem in data]

radiants, dires, targets = [], [], []

for i, elem in enumerate(data):
    pb = elem.picks_bans
    target = elem.pro_match_data.radiant_win

    if pb is None or target is None:
        continue
    
    pb = sorted(filter(lambda x: x.is_pick, pb), key=lambda x: x.team)
    assert len(pb) == 10

    radiants.append([e.hero_id for e in pb[:5]])
    dires.append([e.hero_id for e in pb[5:]])
    targets.append(int(target))

    if 138 in radiants[-1] or 138 in dires[-1]:
        print(i)

len(targets)

18
23
59
90
153
180
208
237
318
319
320
357
397
400
408
417
422
487
520
530
548
556
588
614
639
669
779
832
844
847
871
879
887
912
928
958
984
988


989

In [62]:
torch.tensor(dires, dtype=torch.int32).max()

tensor(138, dtype=torch.int32)

In [64]:
df = pd.DataFrame()
df['target'] = targets
df['radiant_hero_ids'] = radiants
df['dire_hero_ids'] = dires

display(df.sample(10))
df.groupby('target')['target'].count()

,target,radiant_hero_ids,dire_hero_ids
462,0,"[20, 25, 98, 120, 121]","[86, 96, 31, 11, 48]"
933,1,"[20, 101, 96, 4, 28]","[120, 121, 66, 73, 29]"
811,1,"[105, 86, 38, 106, 11]","[5, 71, 119, 41, 22]"
834,1,"[20, 101, 135, 48, 7]","[5, 71, 96, 35, 100]"
627,0,"[19, 5, 76, 62, 107]","[20, 75, 135, 63, 97]"
591,0,"[30, 41, 105, 74, 129]","[100, 64, 106, 43, 12]"
386,1,"[86, 26, 135, 93, 114]","[20, 106, 9, 80, 73]"
742,0,"[20, 26, 19, 48, 23]","[119, 121, 25, 28, 128]"
12,1,"[26, 20, 29, 89, 92]","[111, 69, 112, 41, 39]"
929,1,"[121, 49, 68, 48, 106]","[31, 96, 105, 21, 114]"


target
0    481
1    508
Name: target, dtype: int64

In [70]:
df.iloc[591]

target                                   0
radiant_hero_ids    [30, 41, 105, 74, 129]
dire_hero_ids       [100, 64, 106, 43, 12]
Name: 591, dtype: object

In [74]:
import dataclasses
import json

with open('c.json', 'w') as f:
    j = dataclasses.asdict(data[588])
    f.write(json.dumps(j))

# TODO:

### Разобраться какого хуя id 0 - 138

In [58]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

# df = 
df.join(
    pd.DataFrame.sparse.from_spmatrix(
        mlb.fit_transform(df.pop('radiant_hero_ids')),
        index=df.index,
        columns=mlb.classes_
    )
)

,target,dire_hero_ids,1,2,3,4,5,6,7,8,...,120,121,123,126,128,129,135,136,137,138
0,0,"[121, 126, 21, 105, 135]",0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,"[53, 128, 70, 106, 129]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,"[49, 9, 105, 93, 50]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,"[123, 79, 96, 41, 61]",0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,"[121, 49, 19, 54, 21]",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,0,"[31, 19, 26, 11, 112]",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
985,0,"[19, 66, 49, 12, 120]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
986,0,"[45, 31, 54, 28, 19]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
987,0,"[5, 112, 99, 42, 52]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
